In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import warnings
warnings.filterwarnings('ignore')

import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s4e8/sample_submission.csv
/kaggle/input/playground-series-s4e8/train.csv
/kaggle/input/playground-series-s4e8/test.csv


In [2]:
train = pd.read_csv("/kaggle/input/playground-series-s4e8/train.csv")

In [3]:
test = pd.read_csv("/kaggle/input/playground-series-s4e8/test.csv")

# ***PREPROCESSING***

In [4]:
features_to_drop = ['id']
train = train.drop(features_to_drop, axis=1)

In [5]:
test_sub = test.drop(features_to_drop, axis=1)

In [6]:
# Binarize the class labels
train['class'] = train['class'].apply(lambda x: 0 if x == 'e' else 1)

In [7]:
X = train.drop('class', axis=1)
y = train['class']

In [8]:
cat = X.select_dtypes(include='object').columns.tolist()
threshold = 5000

for col in cat: 
    value_counts = X[col].value_counts()
    labels = set(value_counts[value_counts > threshold].index)
    
    def preprocessor(x):
        if x not in labels: 
            return 'other'
        
        return x 
    
    X[col] = X[col].apply(preprocessor)
    test_sub[col] = test_sub[col].apply(preprocessor)

In [9]:
X = pd.get_dummies(X, columns=cat, drop_first=True)
test_sub = pd.get_dummies(test_sub, columns=cat, drop_first=True)

# ***MODEL PERFORMANCE***

In [10]:
X = X.to_numpy()
y = y.to_numpy()
test_sub = test_sub.to_numpy()

In [11]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import matthews_corrcoef

param_list = {'gamma':0.002797208830158843, 'learning_rate':0.2172811209648847, 'max_depth':11, 'reg_lambda':1.0}
model = XGBClassifier() 
model.set_params(**param_list)

folds = 3
skfolds = StratifiedKFold(n_splits=folds)

total_mcc = []

for train_index, test_index in skfolds.split(X, y):
    X_train_folds, y_train_folds = X[train_index], y[train_index]
    X_test_folds, y_test_folds = X[test_index], y[test_index]
    
    model.fit(X_train_folds, y_train_folds)
    y_pred_folds = model.predict(X_test_folds)
    
    mcc = matthews_corrcoef(y_test_folds, y_pred_folds)
    total_mcc.append(mcc)
    
mcc_cv = sum(total_mcc) / len(total_mcc)
mcc_cv

0.9837240496204706

In [12]:
# from xgboost import XGBClassifier
# from sklearn.metrics import make_scorer, matthews_corrcoef
# from skopt import BayesSearchCV
# from skopt.space import Real, Integer

# param_grid = {
#     'max_depth': Integer(1, 12),
#     'learning_rate': Real(1e-5, 1, prior = 'log-uniform'),
#     'gamma': Real(1e-5, 1, prior = 'log-uniform'), 
#     'reg_lambda': Real(1e-5, 1, prior = 'log-uniform')
# }

# model = XGBClassifier(seed=42, param_grid=param_grid)

# grid_search = BayesSearchCV(estimator=model, search_spaces=param_grid, scoring='matthews_corrcoef', cv=3, verbose=3,
#                             return_train_score=True)

# grid_search.fit(X, y)
# grid_search.best_params_

# ***SUBMISSION***

In [13]:
model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.002797208830158843, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.2172811209648847, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=11, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [14]:
pred_class = model.predict(test_sub)

test['class'] = pred_class
test['class'] = test['class'].apply(lambda x: 'e' if x == 0 else 'p')

submission = test[['id', 'class']]

In [15]:
submission.to_csv('/kaggle/working/VER5.csv', index=False)